* https://elasticsearch-dsl.readthedocs.io/en/latest/search_dsl.html
* !pip install elasticsearch-py
* !pip install elasticsearch-dsl

# ES CREATE INDEX

In [ ]:
!curl -X PUT "localhost:9200/real-estate" -H 'Content-Type: application/json' -d'
{
    "settings" : {
        "number_of_shards" : 1
    },
    "mappings" : {
        "_doc" : {
            "properties": {
                "location": {
                    "type": "geo_point"
                }
            }
        }
    }
}

In [2]:
import sys
sys.path.append('../src')
from es_client import *

# WRITE BULK

In [6]:
df =  load_data('41131', 'trade')
# convert pandas dataframe to list of dict
items = df.to_dict('records')
print ('# of items : %s' % (len(items)))
# data = [item.update({'timestamp': datetime.now()}) for item in items]
docs = []
for item in items:
    docs.append({
        '_index': 'real-estate-trade',
        '_type': 'doc',
        '_id': datetime.now(),
        '_source': item
    })
es = ESClient()
es._bulk(docs)

load /home/irteam/blog/real-estate/data/apt-trade/41131/201201.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201202.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201203.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201204.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201205.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201206.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201207.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201208.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201209.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201210.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201211.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201212.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201301.csv ...
load /home/irteam/blog/real-estate/data/apt-trade/41131/201302.csv ...
load /

In [5]:
df =  load_data('41131', 'rent')
# convert pandas dataframe to list of dict
items = df.to_dict('records')
print ('# of items : %s' % (len(items)))
# data = [item.update({'timestamp': datetime.now()}) for item in items]
docs = []
for item in items:
    docs.append({
        '_index': 'real-estate-rent',
        '_type': 'doc',
        '_id': datetime.now(),
        '_source': item
    })
es = ESClient()
es._bulk(docs)

# of items : 12364
(12364, [])


# READ 

In [2]:
import pandas as pd
es = ESClient()

query = {
    "query": {
        "match": { 
            "아파트": "삼정그린뷰"}
    }
}

query = {
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "아파트": "삼정그린뷰"
          }
        },
        {
          "match": {
            "date": "201209"
          }
        }
      ]
    }
  }
}

results = es._query('real-estate', query)
items = results['hits']['hits']
df = pd.DataFrame([x['_source'] for x in items])
print (df)
# es._query('real-estate', {"query": {"match_all": {}}})

     date    거래금액  건축년도     년  법정동    아파트  월      일     전용면적    지번   지역코드  층
0  201209  34,500  2003  2012  수진동  삼정그린뷰  9  11~20  84.9274  4678  41131  5
1  201209  32,550  2003  2012  수진동  삼정그린뷰  9   1~10  84.9274  4678  41131  4
2  201209  34,500  2003  2012  수진동  삼정그린뷰  9  11~20  84.9274  4678  41131  5
3  201209  32,550  2003  2012  수진동  삼정그린뷰  9   1~10  84.9274  4678  41131  4


# ES_Client에 검색 추가

* es._search_by_date
* es._search_by_aptname
* es._search_by_rcode

In [3]:
df = es._search_by_date('real-estate', "201209") 
print (df)

     date    거래금액  건축년도     년  법정동     아파트  월      일   전용면적    지번   지역코드   층
0  201209  41,000  1986  2012  신흥동    주공저층  9   1~10  83.42    10  41131   2
1  201209  34,800  1986  2012  신흥동    주공고층  9  11~20  64.62    10  41131  15
2  201209  26,000  1996  2012  수진동      삼부  9   1~10  59.76  4663  41131   7
3  201209  32,300  2012  2012  단대동  단대푸르지오  9   1~10  59.97   100  41131   1
4  201209  33,200  2012  2012  단대동  단대푸르지오  9  11~20  59.72   100  41131   1
5  201209  33,200  2012  2012  단대동  단대푸르지오  9  11~20  59.85   100  41131   1
6  201209  33,600  2012  2012  단대동  단대푸르지오  9  21~30  59.80   100  41131   2
7  201209  45,000  1986  2012  신흥동    주공고층  9  11~20  84.15    10  41131   4
8  201209  44,800  2012  2012  단대동  단대푸르지오  9  21~30  84.70   100  41131   2
9  201209  43,000  1986  2012  신흥동    주공고층  9  11~20  84.15    10  41131   1


In [5]:
df = es._search_by_aptname('real-estate', "단대푸르지오") 
print (df)

     date    거래금액  건축년도     년  법정동     아파트   월      일   전용면적   지번   지역코드   층
0  201209  47,500  2012  2012  단대동  단대푸르지오   9   1~10  84.70  100  41131   9
1  201209  33,200  2012  2012  단대동  단대푸르지오   9   1~10  59.72  100  41131   1
2  201209  32,300  2012  2012  단대동  단대푸르지오   9  11~20  59.97  100  41131   1
3  201209  44,800  2012  2012  단대동  단대푸르지오   9  21~30  84.70  100  41131   2
4  201210  47,500  2012  2012  단대동  단대푸르지오  10   1~10  84.70  100  41131  15
5  201305  44,800  2012  2013  단대동  단대푸르지오   5   1~10  84.74  100  41131   2
6  201306  43,900  2012  2013  단대동  단대푸르지오   6  11~20  84.70  100  41131   1
7  201307  43,900  2012  2013  단대동  단대푸르지오   7   1~10  84.70  100  41131   1
8  201309  43,500  2012  2013  단대동  단대푸르지오   9   1~10  84.74  100  41131   2
9  201309  44,800  2012  2013  단대동  단대푸르지오   9  21~30  84.74  100  41131   2


In [6]:
df = es._search_by_rcode('real-estate', 41131) 
print (df)

     date    거래금액  건축년도     년  법정동        아파트  월      일     전용면적      지번  \
0  201201  23,500  1986  2012  신흥동        통보8  1   1~10  67.5000    89-1   
1  201201  33,500  1993  2012  신흥동         두산  1  11~20  84.4700    2024   
2  201201  29,800  2005  2012  태평동    낙원스카이뷰2  1  11~20  84.8400    6146   
3  201201  28,500  2006  2012  수진동       밀란체2  1  11~20  84.8334    4002   
4  201201  28,800  1996  2012  수진동         삼부  1  21~31  59.7600    4663   
5  201201  11,700  1991  2012  단대동       선경논골  1  21~31  34.0500       6   
6  201202  25,400  2007  2012  신흥동       셀레스빌  2  11~20  76.8760   125-2   
7  201202  36,900  1986  2012  신흥동       주공고층  2  11~20  61.5200      10   
8  201202  25,000  1990  2012  신흥동         한신  2  21~29  60.4800  2463-1   
9  201202  24,600  2006  2012  신흥동  삼부르네상스파크1  2  21~29  72.5800     179   

    지역코드   층  
0  41131   1  
1  41131  12  
2  41131  14  
3  41131   4  
4  41131  12  
5  41131  10  
6  41131  13  
7  41131   7  
8  41131  12  
9  41131   4 

# Multi Match Query

In [6]:
from elasticsearch_dsl import Q
q = Q('bool', must=[Q('match', 아파트='단대푸르지오'), Q('match', date='201209')]) 
df = es._search_by_query('real-estate', q) 
print (df)

     date    거래금액  건축년도     년  법정동     아파트  월      일   전용면적   지번   지역코드  층
0  201209  47,500  2012  2012  단대동  단대푸르지오  9   1~10  84.70  100  41131  9
1  201209  33,200  2012  2012  단대동  단대푸르지오  9   1~10  59.72  100  41131  1
2  201209  32,300  2012  2012  단대동  단대푸르지오  9  11~20  59.97  100  41131  1
3  201209  44,800  2012  2012  단대동  단대푸르지오  9  21~30  84.70  100  41131  2
4  201209  33,200  2012  2012  단대동  단대푸르지오  9   1~10  59.72  100  41131  1
5  201209  32,300  2012  2012  단대동  단대푸르지오  9   1~10  59.97  100  41131  1
6  201209  33,200  2012  2012  단대동  단대푸르지오  9  11~20  59.85  100  41131  1
7  201209  35,100  2012  2012  단대동  단대푸르지오  9  11~20  59.72  100  41131  4
8  201209  33,600  2012  2012  단대동  단대푸르지오  9   1~10  59.72  100  41131  2
9  201209  44,800  2012  2012  단대동  단대푸르지오  9  11~20  84.87  100  41131  2
